# This is a live notebook

That means, you can execute the cells -- for example with CTRL+Enter, or click the play button 
above. To start the presentation, click the button as indicated on the following image.

![](assets/explain.png)

In [1]:
#include <xtensor/xio.hpp>
#include <xtensor/xarray.hpp>
#include <xtensor/xtensor.hpp>

![](assets/xtensor.svg)

# xtensor for Robotics

### About me

#### Wolf Vollprecht

- Finished my master in Robotics at ETH last year with Prof. Andreas Krause
- Visiting Stanford (Prof. Marco Pavone) for master thesis: <br>
  »Predicting Human Driver Behavior for Planning under Uncertainty for Autonomous Cars«
- Core developer of xtensor, working at QuantStack (Paris) now
- Find me online:
    - github: @wolfv
    - twitter: @wuoulf
    - email: w.vollprecht@gmail.com

### What's this contraption?

- jupyter notebook with xeus-cling
- cling, the interactive C++ interpreter from CERN
- xeus, the C++ jupyter backend from QuantStack
- more about interactive computing with C++ on the Project Jupyter blog (also plotting!): [Interactive workflows for C++ with Jupyter](https://blog.jupyter.org/interactive-workflows-for-c-with-jupyter-fe9b54227d92)

In [2]:
std::vector<int> random_nums = {1, 1, 2, 3, 5, 8};

In [11]:
// interactive C++!
std::cout << "Interactive EMOJIS! ";
for (const auto& c : random_nums)
    std::cout << c << " 🤘 ";

Interactive EMOJIS! 1 🤘 1 🤘 2 🤘 3 🤘 5 🤘 8 🤘 

### xtensor is ...

- A high-performance nD-array computing library
- Written in modern C++ 14
- Utilizing template metaprogramming magic for compile-time optimizations
- BSD-3 License (all of our projects are!)

### xtensor is ... NumPy in C++

- Trying to follow NumPy's API as close as possible
- Adding *lazyness*
- And speed...

In [4]:
xt::xarray<double> arr = xt::arange<double>(12);
arr.reshape({3, 4});
std::cout << arr << std::endl;

{{  0.,   1.,   2.,   3.},
 {  4.,   5.,   6.,   7.},
 {  8.,   9.,  10.,  11.}}


In [5]:
xt::xtensor<double, 2> brr = arr;
auto crr = arr * brr;
std::cout << crr << std::endl;

{{   0.,    1.,    4.,    9.},
 {  16.,   25.,   36.,   49.},
 {  64.,   81.,  100.,  121.}}


In [6]:
// Broadcasting, NumPy style!
xt::xarray<double> broadcast_moi = {1, 2, 3, 4};
xt::xarray<double> res1 = arr + broadcast_moi;
std::cout << res1 << std::endl;

{{  1.,   3.,   5.,   7.},
 {  5.,   7.,   9.,  11.},
 {  9.,  11.,  13.,  15.}}


### "Lazy" xexpressions

- Through template meta-programming, create an expression tree

```c++
auto expr = (a + b - c) * d;
```

- `expr(1, 1) ==> (a(1, 1) + b(1, 1) - c(1, 1)) * d(1, 1)`
- `xarray<double> res = expr ==> single for-loop!`
  instead of 
  ```
  tmp = (a + b);
  tmp2 = tmp - c;
  tmp3 = tmp2 * d;
  ...
  ```


### xtensor cheatsheet

|    Python 3 - numpy    |     C++ 14 - xtensor    |
|:-----------------------|:------------------------|
| `np.absolute(a)`       | `xt::abs(a)`            |
| `np.sign(a)`           | `xt::sign(a)`           |
| `np.remainder(a, b)`   | `xt::remainder(a, b)`   |
| `np.clip(a, min, max)` | `xt::clip(a, min, max)` |


### Reductions
|      Python 3 - numpy      |    C++ 14 - xtensor    |
|:---------------------------|:-----------------------|
| ``np.sum(a, axis=[0, 1])`` | ``xt::sum(a, {0, 1})`` |
| ``np.sum(a)``              | ``xt::sum(a)``         |
| ``np.prod(a)``             | ``xt::prod(a)``        |
| ``np.mean(a)``             | ``xt::mean(a)``        |

### Mathematical functions

| Python 3 - numpy | C++ 14 - xtensor |
|:-----------------|:-----------------|
| ``np.exp(a)``    | ``xt::exp(a)``   |
| ``np.expm1(a)``  | ``xt::expm1(a)`` |
| ``np.log(a)``    | ``xt::log(a)``   |
| ``np.log1p(a)``  | ``xt::log1p(a)`` |

### Logical functions 
|            Python 3 - numpy                   |                C++ 14 - xtensor               |
|:----------------------------------------------|:----------------------------------------------|
| ``np.where(a > 5, a, b)``                     | ``xt::where(a > 5, a, b)``                    |
| ``np.where(a > 5)``                           | ``xt::where(a > 5)``                          |
| ``np.any(a)``                                 | ``xt::any(a)``                                |
| ``np.all(a)``                                 | ``xt::all(a)``                                |
| ``np.logical_and(a, b)``                      | ``a && b``                                    |
| ``np.logical_or(a, b)``                       | ``a || b``                                    |
| ``np.isclose(a, b)``                          | ``xt::isclose(a, b)``                         |
| ``np.allclose(a, b)``                         | ``xt::allclose(a, b)``                        |


![](assets/cheatsheet.gif)

### Views

- very similar to NumPy

In [7]:
#include <xtensor/xview.hpp>
using namespace xt::placeholders;
std::cout << arr << std::endl;
#arr[::2, :, np.newaxis]
auto va = xt::view(arr, xt::range(_, _, 2), xt::all(), xt::newaxis());
std::cout << "\nThe view\n\n" << va << std::endl;
va += 10;

std::cout << "\nThe underlying array\n\n" << arr << std::endl;

{{  0.,   1.,   2.,   3.},
 {  4.,   5.,   6.,   7.},
 {  8.,   9.,  10.,  11.}}

The view

{{{  0.},
  {  1.},
  {  2.},
  {  3.}},
 {{  8.},
  {  9.},
  { 10.},
  { 11.}}}

The underlying array

{{ 10.,  11.,  12.,  13.},
 {  4.,   5.,   6.,   7.},
 { 18.,  19.,  20.,  21.}}


In [8]:
#include <xtensor/xindex_view.hpp>

auto fa = xt::filter(arr, arr < 10);
std::cout << fa << std::endl;

fa -= 10;

std::cout << "\n\nOriginal array\n\n" << arr << std::endl;

{ 4.,  5.,  6.,  7.}


Original array

{{ 10.,  11.,  12.,  13.},
 { -6.,  -5.,  -4.,  -3.},
 { 18.,  19.,  20.,  21.}}


### How to get it?

- On github: https://github.com/QuantStack/xtensor
- Easiest installation through conda package manager:
    - `conda install xtensor -c conda-forge`
- Or system wide installation through git-cloning and `cmake` / `make install`


### An ecosystem of scientific computing

https://github.com/QuantStack/xtensor-blas

- `xtensor-blas` — high speed `xt::linalg::dot`, `inv`, `eig`, `lstsq`, `svd`...
- using LAPACK and any BLAS (OpenBLAS, MKL, ...) 
- same broadcasting rules for `dot` as NumPy

In [9]:
#include <xtensor-blas/xlinalg.hpp>

xt::xarray<double> a = {{1,2,3}, {4,5,6}, {7,8,9}};
xt::xarray<double> b = {1,2,3};

std::cout << "Dot Product a * b: \n\n" << xt::linalg::dot(a, b) << std::endl;
std::cout << "\nInverse: \n\n" << xt::linalg::inv(a) << std::endl;

Dot Product a * b: 

{ 14.,  32.,  50.}

Inverse: 

{{-4.503600e+15,  9.007199e+15, -4.503600e+15},
 { 9.007199e+15, -1.801440e+16,  9.007199e+15},
 {-4.503600e+15,  9.007199e+15, -4.503600e+15}}


### File input & output: `xtensor-io`

https://github.com/QuantStack/xtensor-io

- xtensor "native": `csv`, `npy`
- xtensor-io
    - xaudio.hpp: `wav`, `flac` (using libsndfile, soon switching to ni-media for mp3 etc.)
    - ximage.hpp: `png`, `jpg`, `bmp`, `gif` ... (using OpenImageIO)
    - xnpz.hpp: `npz` (zipped npy files)
    - xhdf5.hpp: `hdf5` files (using EPFL BlueBrain HighFive library)

```cpp
auto arr = xt::load_image("test.png");

int freq = 2000;
int sampling_freq = 44100;

auto t = xt::arange(0.0, 1.0, 1.0 / sampling_freq);
auto y = xt::sin(2.0 * numeric_constants<double>::PI * freq * t);

xt::dump_audio("files/sine.wav", y, sampling_freq);
```

### Fast fourier transform: `xtensor-fftw`

[https://github.com/egpbos/xtensor-fftw](https://github.com/egpbos/xtensor-fftw)

- Using the extremely fast `fftw` library
- Interface like `numpy.fft`

```c++
double dx = xt::numeric_constants<double>::PI / 100.;
xt::xarray<double> x = xt::arange(0., 2. * xt::numeric_constants<double>::PI, dx);
xt::xarray<double> sin = xt::sin(x);

auto sin_fs = xt::fftw::rfft(sin);
```

### Seamless bindings to Python, R and Julia

https://github.com/QuantStack/xtensor-python

- `xtensor-python`: bind C++ xtensor functions to Python / NumPy
- using the awesome `pybind11`

##### C++

```c++
double sum_of_sines(xt::pyarray<double>& m)
{
    auto sines = xt::sin(m);  // sines does not actually hold values.
    return std::accumulate(sines.begin(), sines.end(), 0.0);
}

PYBIND11_PLUGIN(xtensor_python_test)
{
    xt::import_numpy();
    pybind11::module m("xtensor_python_test", "Test module for xtensor python bindings");

    m.def("sum_of_sines", sum_of_sines, "Sum the sines of the input values");

    return m.ptr();
}
```

-------

##### Python

```python
import numpy as np
import xtensor_python_test as xt

v = np.arange(15).reshape(3, 5)
print(xt.sum_of_sines(v)) #--> 1.2853996391883833 
```

##### C++

```c++
double scalar_func(double i, double j)
{
    return std::sin(i) - std::cos(j);
}

PYBIND11_PLUGIN(xtensor_python_test)
{
    py::module m("xtensor_python_test", "Test module for xtensor python bindings");
    m.def("vectorized_func", xt::pyvectorize(scalar_func), "");
    return m.ptr();
}
```

-------

##### Python

```python
import numpy as np
import xtensor_python_test as xt

x = np.arange(15).reshape(3, 5)
y = [1, 2, 3, 4, 5]
print(xt.vectorized_func(x, y)) #--> [[-0.540302,  1.257618,  1.89929 ,  0.794764, -1.040465],
                                #     [-1.499227,  0.136731,  1.646979,  1.643002,  0.128456],
                                #     [-1.084323, -0.583843,  0.45342 ,  1.073811,  0.706945]]
```

### Secret sauce for speed 👽

- Explicit SIMD vectorization of expression (SSE, AVX)
- Specialized (independent!) package: xsimd (https://github.com/QuantStack/xsimd)
- Also: overloads for SIMD enabled `sin`/`exp`/`errfct` ... (many more)
- E.g. sin ~ 6 times faster than `std::sin`. Generally between 3 and 8 times faster

### Benchmarks 🚀

https://github.com/wolfv/xtensor-benchmark

- xtensor-benchmarks: easy to install & use bench suite
- utilizing google-benchmark
- comparing: Blitz++, Armadillo, Eigen3, xtensor + xsimd

### Benchmarks

Simple operation `array c = a + b;`<br>
Allows debugging against best-in-class librarys

<tbody style="font-family: Overpass;"><tr>
		<td colspan="6" rowspan="2" align="center" height="43" style="font-size: 1.5em; font-family: Overpass; text-align: center;">1D Benchmarks (ns)</td>
		</tr>
	<tr>
		<td align="left" height="21"><br></td>
		<td align="left"><br></td>
		<td align="left"><br></td>
		<td align="left"><br></td>
		<td align="left"><br></td>
		<td align="left"><br></td>
	</tr>
	<tr>
		<td style="border-bottom: 1px solid #000000" align="left" height="21"><font face="Overpass"><br></font></td>
		<td style="border-bottom: 1px solid #000000" align="center"><b><font face="Overpass">Eigen</font></b></td>
		<td style="border-bottom: 1px solid #000000" align="center"><b><font face="Overpass">xtensor</font></b></td>
		<td style="border-bottom: 1px solid #000000" align="center"><b><font face="Overpass">blitz++</font></b></td>
		<td style="border-bottom: 1px solid #000000" align="center"><b><font face="Overpass">Armadillo</font></b></td>
		<td style="border-bottom: 1px solid #000000" align="center"><b><font face="Overpass">xsimd</font></b></td>
	</tr>
	<tr>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="3" sdnum="1033;" align="right" height="21"><b><font face="Overpass">3</font></b></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="19.9219" sdnum="1033;0;0.00" align="right"><font face="Overpass">19.92</font></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="21.4081" sdnum="1033;0;0.00" align="right"><font face="Overpass">21.41</font></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="116.668" sdnum="1033;0;0.00" align="right"><font face="Overpass">116.67</font></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="7.26113" sdnum="1033;0;0.00" bgcolor="#E0EFD4" align="right"><font face="Overpass">7.26</font></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="16.9441" sdnum="1033;0;0.00" align="right"><font face="Overpass">16.94</font></td>
	</tr>
	<tr>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="8" sdnum="1033;" align="right" height="21"><b><font face="Overpass">8</font></b></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="14.7837" sdnum="1033;0;0.00" align="right"><font face="Overpass">14.78</font></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="18.9315" sdnum="1033;0;0.00" align="right"><font face="Overpass">18.93</font></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="119.2" sdnum="1033;0;0.00" align="right"><font face="Overpass">119.20</font></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="9.25907" sdnum="1033;0;0.00" bgcolor="#E0EFD4" align="right"><font face="Overpass">9.26</font></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="14.6707" sdnum="1033;0;0.00" align="right"><font face="Overpass">14.67</font></td>
	</tr>
	<tr>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="64" sdnum="1033;" align="right" height="21"><b><font face="Overpass">64</font></b></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="23.4705" sdnum="1033;0;0.00" bgcolor="#E0EFD4" align="right"><font face="Overpass">23.47</font></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="33.7607" sdnum="1033;0;0.00" align="right"><font face="Overpass">33.76</font></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="133.216" sdnum="1033;0;0.00" align="right"><font face="Overpass">133.22</font></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="40.9595" sdnum="1033;0;0.00" align="right"><font face="Overpass">40.96</font></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="23.098" sdnum="1033;0;0.00" align="right"><font face="Overpass">23.10</font></td>
	</tr>
	<tr>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="512" sdnum="1033;" align="right" height="21"><b><font face="Overpass">512</font></b></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="163.486" sdnum="1033;0;0.00" bgcolor="#E0EFD4" align="right"><font face="Overpass">163.49</font></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="239.119" sdnum="1033;0;0.00" align="right"><font face="Overpass">239.12</font></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="322.97" sdnum="1033;0;0.00" align="right"><font face="Overpass">322.97</font></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="231.585" sdnum="1033;0;0.00" align="right"><font face="Overpass">231.59</font></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="160.999" sdnum="1033;0;0.00" align="right"><font face="Overpass">161.00</font></td>
	</tr>
	<tr>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="1000" sdnum="1033;" align="right" height="21"><b><font face="Overpass">1000</font></b></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="237.917" sdnum="1033;0;0.00" bgcolor="#E0EFD4" align="right"><font face="Overpass">237.92</font></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="338.206" sdnum="1033;0;0.00" align="right"><font face="Overpass">338.21</font></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="484.751" sdnum="1033;0;0.00" align="right"><font face="Overpass">484.75</font></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="391.06" sdnum="1033;0;0.00" align="right"><font face="Overpass">391.06</font></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="227.622" sdnum="1033;0;0.00" align="right"><font face="Overpass">227.62</font></td>
	</tr>
</tbody>

<tbody>	
    <tr>
		<td colspan="6" rowspan="2" align="center" height="43" style="font-size: 1.5em; font-family: Overpass; text-align: center;">2D Benchmarks (ns)</td>
		</tr>
	<tr>
		<td align="left" height="21"><br></td>
		<td align="left"><br></td>
		<td align="left"><br></td>
		<td align="left"><br></td>
		<td align="left"><br></td>
		<td align="left"><br></td>
	</tr>
	<tr>
		<td style="border-bottom: 1px solid #000000" align="left" height="21"><font face="Overpass"><br></font></td>
		<td style="border-bottom: 1px solid #000000" align="center"><b><font face="Overpass">Eigen</font></b></td>
		<td style="border-bottom: 1px solid #000000" align="center"><b><font face="Overpass">xtensor</font></b></td>
		<td style="border-bottom: 1px solid #000000" align="center"><b><font face="Overpass">blitz++</font></b></td>
		<td style="border-bottom: 1px solid #000000" align="center"><b><font face="Overpass">Armadillo</font></b></td>
		<td style="border-bottom: 1px solid #000000" align="center"><b><font face="Overpass">xsimd</font></b></td>
	</tr>
	<tr>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="3" sdnum="1033;" align="right" height="21"><b><font face="Overpass">3</font></b></td>
		<td style="border-bottom: 1px solid #000000" sdval="29.0644" sdnum="1033;0;# ##0.00" align="right"><font face="Overpass"> 29.06</font></td>
		<td style="border-bottom: 1px solid #000000" sdval="24.3564" sdnum="1033;0;# ##0.00" align="right"><font face="Overpass"> 24.36</font></td>
		<td style="border-bottom: 1px solid #000000" sdval="191.131" sdnum="1033;0;# ##0.00" align="right"><font face="Overpass"> 191.13</font></td>
		<td style="border-bottom: 1px solid #000000" sdval="13.5615" sdnum="1033;0;# ##0.00" bgcolor="#E0EFD4" align="right"><font face="Overpass"> 13.56</font></td>
		<td style="border-bottom: 1px solid #000000" sdval="18.9505" sdnum="1033;0;# ##0.00" align="right"><font face="Overpass"> 18.95</font></td>
	</tr>
	<tr>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="8" sdnum="1033;" align="right" height="21"><b><font face="Overpass">8</font></b></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="34.2939" sdnum="1033;0;# ##0.00" bgcolor="#E0EFD4" align="right"><font face="Overpass"> 34.29</font></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="34.4573" sdnum="1033;0;# ##0.00" bgcolor="#E0EFD4" align="right"><font face="Overpass"> 34.46</font></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="166.262" sdnum="1033;0;# ##0.00" align="right"><font face="Overpass"> 166.26</font></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="57.544" sdnum="1033;0;# ##0.00" align="right"><font face="Overpass"> 57.54</font></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="23.2297" sdnum="1033;0;# ##0.00" align="right"><font face="Overpass"> 23.23</font></td>
	</tr>
	<tr>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="64" sdnum="1033;" align="right" height="21"><b><font face="Overpass">64</font></b></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="1363.97" sdnum="1033;0;# ##0.00" bgcolor="#E0EFD4" align="right"><font face="Overpass">1 363.97</font></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="1775.82" sdnum="1033;0;# ##0.00" align="right"><font face="Overpass">1 775.82</font></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="3018.06" sdnum="1033;0;# ##0.00" align="right"><font face="Overpass">3 018.06</font></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="2321.3" sdnum="1033;0;# ##0.00" align="right"><font face="Overpass">2 321.30</font></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="1340.55" sdnum="1033;0;# ##0.00" align="right"><font face="Overpass">1 340.55</font></td>
	</tr>
	<tr>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="512" sdnum="1033;" align="right" height="21"><b><font face="Overpass">512</font></b></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="288917" sdnum="1033;0;# ##0.00" align="right"><font face="Overpass">288 917.00</font></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="246587" sdnum="1033;0;# ##0.00" bgcolor="#E0EFD4" align="right"><font face="Overpass">246 587.00</font></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="1025800" sdnum="1033;0;# ##0.00" align="right"><font face="Overpass">1025 800.00</font></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="264378" sdnum="1033;0;# ##0.00" align="right"><font face="Overpass">264 378.00</font></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="78483.7" sdnum="1033;0;# ##0.00" align="right"><font face="Overpass">78 483.70</font></td>
	</tr>
	<tr>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="1000" sdnum="1033;" align="right" height="21"><b><font face="Overpass">1000</font></b></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="1683060" sdnum="1033;0;# ##0.00" align="right"><font face="Overpass">1683 060.00</font></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="2949090" sdnum="1033;0;# ##0.00" align="right"><font face="Overpass">2949 090.00</font></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="1487640" sdnum="1033;0;# ##0.00" align="right"><font face="Overpass">1487 640.00</font></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="1329600" sdnum="1033;0;# ##0.00" bgcolor="#E0EFD4" align="right"><font face="Overpass">1329 600.00</font></td>
		<td style="border-top: 1px solid #000000; border-bottom: 1px solid #000000" sdval="525787" sdnum="1033;0;# ##0.00" align="right"><font face="Overpass">525 787.00</font></td>
	</tr>
</tbody>

### Results

- Eigen3 definitly the fastest traditional library
- Armadillo really fast for small matrices/vectors
- xtensor with compile-time known dimensions comparably fast as Eigen::MatrixXd
- Let's see what we can learn ... 

### Armadillo tricks

- pre-allocate small amount of storage on the stack
- eliminates cost of allocating fixed N of elements at runtime
- huge benefit for small arrays

➡ same idea in xtensor small_vector, soon to be merged <br>

### Eigen tricks

- most people use fixed size Eigen containers (e.g. Matrix3f)
- soon in xtensor: `xfixed<double, <3, 1, 5>>`
- calculate storage size/strides at compile time, allocate on stack

### xtensor for robotics

- many problems in robotics require 1, 2, or N-D arrays
- Armadillo and Eigen dominant libraries in the robotics space
- Limited amount of dimensions, no broadcasting, "legacy" code

(still, Eigen is really fast!)

![](assets/xtensor-ros.svg)

### ROS support

- native support to send and receive xtensor messages
- as few copies as possible
- (soon) native communication with NumPy over msgs
- (soon) update for ROS 2

### Code example

```c++
ros::Publisher chatter_pub = n.advertise<xt::xarray<double>>("chatter", 1000);

// .... 

ros::Subscriber sub = n.subscribe("chatter", 1000, chatterCallback);

void chatterCallback(const xt::xarray<double>& msg_arr)
{
    ROS_INFO("I heard: something");
    std::cout << msg_arr << std::endl;
}
```

# Thanks

### Questions or comments?

Bonus: check out these interactive notebooks online (Binder is awesome)

- https://mybinder.org/v2/gh/egpbos/xtensor-fftw/master?filepath=notebooks%2Fintensely_edgy_cat.ipynb
- https://mybinder.org/v2/gh/QuantStack/xtensor-io/master?filepath=notebooks/demo.ipynb
- https://beta.mybinder.org/v2/gh/QuantStack/xtensor/0.14.1-binder?filepath=notebooks/xtensor.ipynb

### Join us on gitter! https://gitter.im/quantstack